# K-Nearest Neighbors

In [2]:
# Manipulação dos dados
import numpy  as np
import scipy  as sp
import pandas as pd

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn           as sns
%matplotlib inline

In [3]:
# Carregando os dados
data = pd.read_csv('../../Dados/preprocessed.csv')

In [5]:
data.head(5)

,desfecho,TTO_anterior_TB_triagem_enfermeiro,Cicatriz_BCG,Idade,Sexo,Raça,Estado_civil,Tem_companheiro,Semanas_tosse,Semanas_expectoração,...,Dispnéia,Perda_de_apetite,Perda_peso_10percent,Dor_torácica,Internação_hospital_2anos,Prisão_2anos,Delegacia_2anos,Contato_TBP_2anos,Fuma,CAGE
0,TB+,Sim,Sim,30,Masculino,Não branco,Solteiro,Não,1,0,...,Não,Não,Não,Não,Sim,Não,Não,Não,Fumante,Negativo
1,TB+,Sim,ignorado,53,Feminino,Branco,ignorado,Não,0,0,...,Sim,ignorado,Não,Não,Não,Não,Não,Não,Ex-fumante,Negativo
2,TB+,Não,Não,84,Feminino,Branco,viúvo,Não,1,1,...,Não,ignorado,Sim,Sim,Não,Não,Não,Não,Jamais fumante,Negativo
3,TB+,Não,Sim,18,Feminino,Branco,Solteiro,Sim,1,0,...,Sim,Não,Não,Sim,Não,Não,Não,Sim,Jamais fumante,Negativo
4,TB+,Sim,ignorado,36,Masculino,Não branco,Solteiro,Não,0,0,...,Sim,Não,Não,Sim,Não,Não,Não,Não,Fumante,Positivo


In [5]:
# Importando o KNN do scikit-learn 
from sklearn.neighbors import KNeighborsClassifier as KNN

In [26]:
# Atributos considerados pelo WHO
features_selecionadas = ['desfecho','Semanas_expectoração','Semanas_tosse','Hemoptise','Dor_torácica',
                         'Perda_peso_10percent','Semanas_sudorese','Semanas_febre']

# Dataframe com os dados
dados = data[features_selecionadas].copy()

In [70]:
dados.desfecho.value_counts()/len(dados)

TB-    0.680601
TB+    0.319399
Name: desfecho, dtype: float64

In [71]:
dados.desfecho.value_counts()

TB-    1856
TB+     871
Name: desfecho, dtype: int64

## Modelo 1 : WHO

In [30]:
# Modelo conservador: Ignorando qualquer dado que tenha o valor "ignorado"

# Alterando os valores para nan  
dados.replace(['ignorado'],[np.nan],inplace=True)

# Quantas linhas possuem valores ignorados
dados_modelo = dados.dropna(how='any')

In [34]:
dados_modelo.head(5)

,desfecho,Semanas_expectoração,Semanas_tosse,Hemoptise,Dor_torácica,Perda_peso_10percent,Semanas_sudorese,Semanas_febre
0,TB+,0,1,Não,Não,Não,0,0
1,TB+,0,0,Não,Não,Não,0,0
2,TB+,1,1,Não,Sim,Sim,0,1
3,TB+,0,1,Não,Sim,Não,0,0
4,TB+,0,0,Não,Sim,Não,0,0


In [39]:
# 
from sklearn.preprocessing   import LabelEncoder
from sklearn.model_selection import cross_val_score

dados_modelo = dados_modelo.apply(LabelEncoder().fit_transform)

# K= 5
clf = KNN()

y = dados_modelo.desfecho
X = dados_modelo.drop(['desfecho'],axis=1)

In [44]:
print('AUROC: {:.2f} +/- {:.2f}'.format(np.mean(cross_val_score(clf,X,y,cv=10,scoring='roc_auc')),
                                        np.std(cross_val_score(clf,X,y,cv=10,scoring='roc_auc'))))

AUROC: 0.55 +/- 0.06


In [68]:
# 
dados.replace([np.nan],'ignorado',inplace=True)

# Abordagem utilizando One-hot encoding
y = dados.desfecho
y = y.replace(['TB+','TB-'],[1,0])
X = pd.get_dummies(dados.drop(['desfecho'],axis=1))

In [69]:
print('AUROC: {:.2f} +/- {:.2f}'.format(np.mean(cross_val_score(clf,X,y,cv=10,scoring='roc_auc')),
                                        np.std(cross_val_score(clf,X,y,cv=10,scoring='roc_auc'))))

AUROC: 0.59 +/- 0.05


## Modelo 2 : CHI2

In [ ]:
import pickle

## Modelo 4 : Regressão Logística L1

### Considerando dados ignorados

### One-hot Encoding

## Modelo 5 : Random Forests

### Considerando dados ignorados

### One-hot Encoding